# Entrenamiento de un modelo de lengua extremadamente simple

<a target="_blank" href="https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/assets/misterios/notebooks/simple-lm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="http://dlsi.ua.es/~japerez/"><img src="https://img.shields.io/badge/Universitat-d'Alacant-5b7c99" style="margin-left:10px"></a>

Cuaderno y código escrito por Juan Antonio Pérez en 2025. Originalmente inspirado en el código de Tae Hwan Jung (@graykode) en el [tutorial de NLP](https://github.com/graykode/nlp-tutorial).

Este cuaderno presenta un modelo de lengua intencionadamente simple basado en unas redes neuronales llamadas *feed-forward neural networks* o también *perceptrones multicapa*. Está tan simplificado que será entrenado con oraciones muy cortas y solo podrá predecir la siguiente palabra a partir de las dos palabras anteriores. Además, nuestro conjunto de entrenamiento será tan pequeño que el modelo probablemente lo memorizará. Esto implica que la red no podrá generalizar a oraciones que no haya visto durante el entrenamiento y, como resultado, la probaremos con las mismas oraciones utilizadas para entrenar; esto constituye una muy mala práctica, que utilizaremos aquí únicamente por razones de simplicidad. A pesar de todas estas limitaciones, este cuaderno será útil para ilustrar los conceptos básicos del modelado del lenguaje con redes neuronales simples.

Para ejecutar este cuaderno de Python puedes usar un entorno como Google Colab, que te permitirá ejecutar el código en la nube sin necesidad de instalar nada en tu ordenador. Desde el menú *Entorno de ejecución* puedes seleccionar *Ejecutar todas* para ejecutar todas las celdas de código. El consumo de recursos es tan reducido que el entorno de ejecución que uses no necesita GPU: asegúrate en *Entorno de ejecución* / *Cambiar tipo de entorno de ejecución* de que no estás gastando recursos innecesarios y reserva las pocas horas gratuitas de GPU que tienes para tareas más exigentes.

## Configuración del entorno

La siguiente celda configura establece una semilla para el generador de números aleatorios de forma que los resultados sean siempre los mismos entre distintas ejecuciones.

In [ ]:
import os
# set before importing pytorch to avoid all non-deterministic operations on GPU
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import random
import numpy as np
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True)

set_seed(42)  # to ensure reproducibility

## Datos de entrenamiento y vocabulario

La siguiente celda define las frases del conjunto de entrenamiento. El vocabulario se construye a partir de todas las palabras que hay en estas frases. La variable `window_size` define el número de palabras que se usarán para predecir la siguiente palabra, pero no es necesario que la modifiques. La función `make_batch` genera un bloque de datos de entrenamiento en el que las entradas son los enteros que representan las palabras previas y la salida deseada se representa con el índice de la siguiente palabra.

In [ ]:
window_size = 2

sentences = ["let's promote peace", "let's ban war", "let's teach compassion", "let's build a better world"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_index = {w: i for i, w in enumerate(word_list)}
index_word = {i: w for i, w in enumerate(word_list)}
vocab_size = len(word_index)

print(f"word_index = {index_word}")

def make_batch():
    input_batch = []
    target_batch = []

    for sentence in sentences:
        words = sentence.split()  # space tokenizer
        for i in range(len(words) - 2):
            input = [word_index[words[i]], word_index[words[i+1]]]  # indices of two consecutive words
            target = word_index[words[i+2]]  # index of the next word
            input_batch.append(input)
            target_batch.append(target)

    return input_batch, target_batch

inputs, targets = make_batch()
print(f"inputs = {inputs}")
print(f"targets = {targets}")
print(f"inputs = {[[index_word[i] for i in x] for x in inputs]}")
print(f"targets = {[index_word[i] for i in targets]}")

## Definición del modelo

Aunque no vas a entender completamente el código de la siguiente celda, en ella se definen los módulos de la librería PyTorch que componen nuestro modelo. La función `nn.Embedding` permite convertir los índices de las palabras en vectores de embeddings de tamaño `embedding_size`. Nuestro modelo tiene dos matrices creadas por la función `nn.Linear`. La primera, `W`, es con la que se multiplica la entrada para obtener un nuevo vector. Este vector intermedio se multiplica por la segunda matriz, `U`, para obtener los valores de salida.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNLM(nn.Module):
    def __init__(self, vocab_size, embedding_size, window_size, hidden_size):
        super().__init__()
        self.embedding_size = embedding_size
        self.window_size = window_size
        self.C = nn.Embedding(vocab_size, embedding_size)
        self.W = nn.Linear(window_size * embedding_size, hidden_size, bias=True)
        self.U = nn.Linear(hidden_size, vocab_size, bias=True)

    def forward(self, X):
        X = self.C(X)
        X = X.view(-1, self.window_size * self.embedding_size)
        X = F.relu(self.W(X))
        X = self.U(X)
        return X  # return logits

## Entrenamiento del modelo

El siguiente código es muy habitual cuando se entrenan redes neuronales. El modelo se crea en la llamada a `NNLM` (que es la clase que hemos definido antes) y se guarda en la variable `model`. A continuación, se define la función de pérdida en la variable `criterion` y el mecanismo de actualización de los parámetros en la variable `optimizer`.

Después de obtener los datos en la llamada a `make_batch`, el programa entra en un bucle de entrenamiento durante `training_steps` pasos. En cada paso del bucle, se computa la salida de la red neuronal en la variable `output` tras pasar al modelo los datos de entrada guardados en `input_batch`; a continuación, se calcula la pérdida u error entre la salida obtenida en `output` y la salida deseada que está en `target_batch`; después, la llamada a `backward` calcula la información necesaria para determinar el cambio a realizar en cada parámetro, actualización que se lleva efectivamente a cabo en la llamada a `optimizer.step`.

Cada cierto número de pasos, se imprime el valor de la función de pérdida que idealmente se irá reduciendo a medida que el modelo aprenda.

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

hidden_size = 4
embedding_size = 2
training_steps = 1000
eval_steps = 100
lr = 0.005

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = NNLM(vocab_size, embedding_size, window_size, hidden_size)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=training_steps)

input_batch, target_batch = make_batch()
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

lr_history = []  # learning rate history
model.train()
for i in range(training_steps):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    if i % eval_steps == 0:
        print(f'Step [{i}/{training_steps}], loss: {loss.item():.4f}')
    lr_history.append(scheduler.get_last_lr()[0])

    loss.backward()
    optimizer.step()
    scheduler.step()  # scheduler must be called after optimizer

print(f'Step [{training_steps}/{training_steps}], loss: {loss.item():.4f}')

## Evaluación del modelo

Evaluamos el modelo con sus propios datos de entrenamiento, lo cual es una práctica terrible, pero la única que tiene algo de sentido dado el tamaño extremadamente pequeño del conjunto de entrenamiento, que no permitirá que el modelo generalice a datos no vistos ni desarrolle predicciones útiles.

En cada línea se imprimen dos palabras y la palabra que según el modelo entrenado tiene más probabilidad de ser la siguiente. También se imprimen las tres continuaciones con probabilidad más alta y el valor de esta.

Prueba a añadir al conjunto de entrenamiento alguna frase que comparta un prefijo de dos palabras con alguna existente (por ejemplo, *let's ban guns*) y observa las probabilidades predichas para la palabra siguiente al prefijo común (es decir, para la continuación de *let's ban...*).

In [ ]:
import torch.nn.functional as F

model.eval()
with torch.no_grad():
    predict = model(input_batch)
    predict_max = predict.argmax(dim=1)

    for i in range(len(input_batch)):
        print([index_word[n.item()] for n in input_batch[i]], '⟶', index_word[predict_max[i].item()], end=' || ')
        top3 = predict[i].topk(3)
        probabilities = F.softmax(top3.values, dim=0)
        formatted_output = ', '.join([f"{index_word[top3.indices[j].item()]}:{probabilities[j].item():.3f}" for j in range(3)])
        print(formatted_output)